# Исследование надёжности заёмщиков

**Цель исследования** 

Проверить:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

4. Как разные цели кредита влияют на его возврат в срок?

**Входные данные** 

Статистика о платёжеспособности клиентов от банка.

**Заказчик исследования**

Кредитный отдел банка. 

**Практическое применение результатов исследования**

Результаты исследования будут учтены при построении модели *кредитного скоринга* — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Обзор данных

In [1]:
# Импортируем библиотеки для проекта
import pandas as pd
from nltk.corpus import stopwords
from pymystem3 import Mystem
mystem = Mystem()

In [2]:
# Прочитаем данные и сохраним в df:
df = pd.read_csv('data.csv') 

In [3]:
# Посмотрим на первые 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# Получим общую информацию о данных в таблице data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



**Описание данных согласно документации:**

* children — количество детей в семье

* days_employed — общий трудовой стаж в днях

* dob_years — возраст клиента в годах

* education — уровень образования клиента

* education_id — идентификатор уровня образования

* family_status — семейное положение

* family_status_id — идентификатор семейного положения

* gender — пол клиента

* income_type — тип занятости

* debt — имел ли задолженность по возврату кредитов

* total_income — ежемесячный доход

* purpose — цель получения кредита


In [5]:
# Посмотрим, есть ли явные дубликаты:
df.duplicated().sum()

54

In [6]:
#Посмотрим на описательную статистику нашего датасета
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<a id='intro'></a>
Рассмотрим уникальные значения данных для остальных столбцов (пол, образование, семейное положение, тип занятости, цель):

In [7]:
for column_name in df.columns:
    if column_name not in df.describe().columns:
        print(df[column_name].value_counts(), '\n')

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64 

свадьба                                 


**Вывод**

<div style="border:solid green 2px; padding: 20px">
    
В таблице 12 столбцов. Общее количество наблюдений в данных 21525. В двух столбцах значений меньше: общий трудовой стаж в днях и ежемесячный доход.

В датафрейме представлены данные разных типов: строка, дробное число, целое число. 
    
В названиях столбцов нарушений нет. 
  
Предварительно можно утверждать, что данных достаточно для проведения исследования. 

**Однако есть следующие проблемы в данных**:
    
1. В столбце children есть артефакты: -1 (встречается 47 раз) и 20 (встречается 76 раз).

2. В столбце days_employed есть пропуски в данных, а также отрицательные значения и тип данных число с плавающей точкой требует преобразования в integer. 
    
3. В столбце dob_years есть артефакты: возраст со значением 0 (встречается 101 раз). 
    
4. В столбце family_status есть значение "Не женат / не замужем", которое требуется привести к строчному регистру по аналогии со всеми остальными.
   
5. В столбце gender есть артефакт: XNA. (Всего одна строка, поэтому можно просто удалить эту строчку.)  
    
6. В столбце total_income есть пропуски в данных, а также тип данных число с плавающей точкой требует преобразования в целочисленный. 
    
7. В столбце education в значениях есть неявные дубликаты. Требуется привести все значения к нижнему регистру. 

8. В данных выявлено 54 явных дубликата. 

Почистим данные - устраним проблемы. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [8]:
# Подсчитаем пропуски
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [9]:
# Вычислим, каков процент пропущенных значений в столбце total_income:
df_nan = df[df["total_income"].isnull()]
print('Значения отсутствуют в {:.1%}'.format(len(df_nan)/len(df)))

Значения отсутствуют в 10.1%


In [10]:
#Посмотрим на первые 5 строк датафрейма со строками с nan
df[df.isnull().any(1)].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [11]:
# Проверим распределение NaN, действительно ли оно одинаковое:
len(df[df["total_income"].isnull() & df["days_employed"].isnull()])

2174

Пропуски в доходе и стаже распределены одинаково. Пропуски случайные. Возможно, данные утеряны в процессе выгрузки данных или изначально отсутствовали. 

#### Обработка пропусков в столбце ежемесячного дохода (total_income) <a class="tocSkip">

In [12]:
df['total_income'].isna().sum()

2174

In [13]:
#Сгруппируем данные по типу занятости и посчитаем медиану ежемесячного дохода для каждого типа. 
total_income_median = df.groupby('income_type')['total_income'].median()
total_income_median

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Заполним пропуски в столбце total_income значеним медианы в зависимости от типа занятости

In [14]:
#Применим метод transform, чтобы на ходу посчитать медиану в группировке и заполнить пропуски с помощью fillna
df['total_income'] = df['total_income'].fillna(df.groupby(['income_type'])['total_income'].transform('median'))

In [15]:
df['total_income'].isna().sum()

0


*Итог*: 

Пропуски в столбце с ежемесячным доходом успешно заполнены значенияем медианы в зависимости от типа занятости. 

#### Обработка пропусков в столбце трудового стажа (days_employed) <a class="tocSkip">


Теперь приступим к подготовке данных для обработки пропущенных значений в столбце с трудовым стажем (days_employed).

Алгоритм действий таков:

1. Заменим возраст клиента 0 на среднее арифметическое. 
2. Категоризуем данные - распределим клиентов по возрастным группам и создадим соответствующий столбец (age_group) с данными в нашем датафрейме.
3. Более внимательно изучим значения в столбце days_employed, избавимся от отрицательных значений и исключим аномалии (выбросы в данных), чтобы более точно определить медиану трудового стажа.
3. Cоздадим series со значениями медианы для каждой возрастной группы и без аномалий.
4. Запустим функцию replace_nan_with_median для заполнения пропусков в столбце days_employed значениями медианы в зависимости от возрастой группы.  

In [16]:
# Возраст - количественная переменная, мы можем нулевой возраст заменить на значение среднего арифметического.
df = df.replace({'dob_years': 0}, df['dob_years'].mean())

In [17]:
# Проверяем, успешна ли замена 0-значений средним арифметическим
df[df['dob_years'] == 0]['dob_years'].count()

0

In [18]:
# Создадим функцию для категоризации данных о возрасте клиентов (распределим клиентов по возрастным группам)

'''
Функция берет на вход возраст клиента, а возвращает возрастную группу по значению возраста age, используя правила:
    - ранний возраст (19 - 30)
    - средний возраст (30 - 40)
    - поздний возраст (40 - 59)
    - пенсионный возраст (60 и старше)

'''

def get_age_group(age):
    if 30 > age >= 19:
        return 'ранний возраст'
    elif 40 > age >=30:
        return 'средний возраст' 
    elif 59 >= age >=40:
        return 'поздний возраст'
    else:
        return 'пенсионный возраст'
        

In [19]:
#Протестируем нашу функцию client_age_group на отдельном значении:
get_age_group(60)

'пенсионный возраст'

In [20]:
# Создадим отдельный столбец age_group в нашем датафрейме и запишем в его ячейках значения, возращаемые функцией
df['age_group'] = df['dob_years'].apply(get_age_group)

In [21]:
# Посмотрим на результаты
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,поздний возраст
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,средний возраст
2,0,-5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,средний возраст
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,средний возраст
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,поздний возраст


Функция отработала корректно

In [22]:
# Теперь заполним пропуски нулями, чтобы корректно посчитать медиану трудового стажа.
df['days_employed'] = df['days_employed'].fillna(0)

In [23]:
#Для начала более внимательно изучим данные о трудовом стаже:
df['days_employed'].describe()

count     21525.000000
mean      56678.874622
std      134870.763085
min      -18388.949901
25%       -2518.168900
50%        -982.531720
75%           0.000000
max      401755.400475
Name: days_employed, dtype: float64

In [24]:
#Посмотрим отдельно на положительные значения: 
df_with_positive_days_employed = df[df['days_employed'] > 0]
print('\nКоличество клиентов с положительным трудовым стажем и тип их занятости:\n',
      df_with_positive_days_employed.groupby('income_type')['days_employed'].count())


Количество клиентов с положительным трудовым стажем и тип их занятости:
 income_type
безработный       2
пенсионер      3443
Name: days_employed, dtype: int64


В диапазоне положительных значений выявлено большое количество аномальных данных. 3443 пенсионера и два безработных имеют стаж более 900 лет (более 50%).
Хотя медиана не чувствительна к выбросами, все равно исключим их из вычисления медианы. Для этого создадим отдельный фрейм, в котором останутся только отрицательные значения и на основе его будет считать медиану для каждой возрастной группы. Удалять выбросы из исходного датафрейма не будем: этот столбец нам не важен для данного исследования, а другие данные из строк терять не хочется. 
Возможно, такие большие значения стажа - это значения в других единицах, не в днях, а в часах.  

In [25]:
#Создаем отдельный датафрейм только с отрицательными значениями в days_employed. 
df_without_outliners_in_days_employed = df[df['days_employed'] <= 0]

In [26]:
#Проверим результат - положительных значений нет. 
df_without_outliners_in_days_employed[df_without_outliners_in_days_employed['days_employed'] > 0]['days_employed'].count()

0

В столбце о трудовом стаже больше половины отрицательных значений. Возможно, при подготовке и вычислении данных разработчики перепутали местами даты начала трудовой деятельности и текущий (или последний) день работы. 
Устраним эту проблему.

In [27]:
#Чтобы избавиться от отрицательных значений в столбце о трудовом стаже (days_employed) в исходном датафрейме, вычислим модуль числа с помощью встроенной функции abs

print('Количество отрицательных значений до обработки:', df[df['days_employed'] < 0]['days_employed'].count())
df['days_employed'] = abs(df['days_employed'])
print('Количество отрицательных значений после обработки:', df[df['days_employed'] < 0]['days_employed'].count())

Количество отрицательных значений до обработки: 15906
Количество отрицательных значений после обработки: 0


In [28]:
#Избавимся от отрицательных значений в датафрейме, в котором нет выбросов, и который мы будем использовать для группировки данных и расчета медианы: 

print('Количество отрицательных значений до обработки:', 
      df_without_outliners_in_days_employed.loc[df_without_outliners_in_days_employed['days_employed'] < 0]['days_employed'].count())

df_without_outliners_in_days_employed.loc[:, 'days_employed'] = abs(df_without_outliners_in_days_employed['days_employed'])

print('Количество отрицательных значений после обработки:', 
      df_without_outliners_in_days_employed.loc[df_without_outliners_in_days_employed['days_employed'] < 0]['days_employed'].count())



Количество отрицательных значений до обработки: 15906
Количество отрицательных значений после обработки: 0


/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [29]:
#Заполним пропуски в столбце о стаже: 
# Сгруппируем данные по возрастным категориям, посчитаем медиану стажа в каждой и через метод transform подадим в fillna
df['days_employed'] = df['days_employed'].fillna(df.groupby(['age_group'])['days_employed'].transform('median'))

In [30]:
# Подсчитаем пропуски в нашем датафрейме - проверим результаты
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

**Вывод**

<div style="border:solid green 2px; padding: 20px">
 
* В столбцах с ежемесячным доходом (total_income) и с трудовым стажем (days_employed) были обнаружены пропущенные значения (NaN). Пропуски в данных составляли 10%. 
    
* Ежемесячный доход и трудовой стаж, измеряемый в днях  — это количественные переменные. 
Пропуски в таких переменных, как правило, заполняют характерными значениями в выборке. Для типичных значений мы взяли **медиану**, потому что она позволяет избежать влияния выбросов в данных при расчетах. 

* Мы успешно заполнили пропуски в столбце total_income значением медианы в зависимости от типа занятости (столбец income_type). 
А в столбце трудового стажа - медианой в зависимости от возрастной группы (столбец age_group, который был создан в процессе заполнения пропусков).
    
* Стоит отметить, что пропущенные значения в трудовом стаже не влияют на результаты данного исследования, однако пропуски мы обработали и в этом столбце, чтобы была возможность использовать данные для возможных дальнейших исследований. 
    
* Возможно, стоит обратить внимание разработчиков на проблему отрицательных значений в days_employed (предположительно, при подготовке и вычислении данных разработчики перепутали местами даты начала трудовой деятельности и текущий (или последний) день работы).

### Замена типа данных

In [31]:
#Посмотрим на типы данных в нашем датафрейме
df.dtypes

children              int64
days_employed       float64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
age_group            object
dtype: object

In [32]:
# Заменим числа с плавающей точкой на целые для столбцов days_employed, dob_years, total_income:
df = df.astype({'days_employed': 'int64', 'dob_years': 'int64', 'total_income': 'int64'})
print('Типы данных в датафрейме после замены:\n', df.dtypes)

Типы данных в датафрейме после замены:
 children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
age_group           object
dtype: object


**Вывод**

<div style="border:solid green 2px; padding: 20px">
В столбцах c трудовым стажем (days_employed), возрастом клиентов (dob_years) и с ежемесячным доходом (total_income) в качестве значений выступали дробные числа (вещественный тип данных float64). Такой тип данных можно изменить на целочисленный (int64) с помощью метода astype, что мы успешно и сделали.

### Обработка артефактов


In [33]:
#В столбце gender есть значение XNA. Всего лишь одно - тут можно просто удалить строку из данных.
print('Количество значений XNA в столбце с полом: ', df[df['gender'] == 'XNA']['gender'].count())
df = df.loc[df['gender'] != 'XNA']
print('Количество значений XNA в столбце с полом после удаления строки: ', df[df['gender'] == 'XNA']['gender'].count())

Количество значений XNA в столбце с полом:  1
Количество значений XNA в столбце с полом после удаления строки:  0


In [34]:
#В столбце с количеством детей 76 раз встречается значение 20 и 47 раз - значение -1. Заменим эти значения на 2 и 1 соответственно.
print(df['children'].value_counts())
df['children'].replace({20: 2, -1: 1}, inplace=True)
print('\nРаспределение значений после замены -1 и 20: \n', df['children'].value_counts())

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Распределение значений после замены -1 и 20: 
 0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [35]:
#В столбце family_status значения записаны в разном регистре - унифицируем здесь заодно. 
df['family_status'].value_counts()


женат / замужем          12380
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [36]:
#Устраним эту проблему с помощью метода str.lower() и проверим результат
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()


женат / замужем          12380
гражданский брак          4176
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Обработка дубликатов

Сначала лучше устранить неявные дубликаты, а потом одним разом удалить все явные. 

В [начале исследования](#intro) мы смотрели на уникальные значения данных для столбцов в датафрейме и обнаружили неявные дубликаты в значениях столбца 'education'.

In [37]:
# Приведем к нижнему регистру все значения в столбце education и тут же проверим результат выполнения метода str.lower():
df['education'] = df['education'].str.lower()
print('Значения в education после применения str.lower:\n', df['education'].value_counts())

Значения в education после применения str.lower:
 среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64


In [38]:
# Посмотрим на количество явных дубликатов с помощью цепочки методов duplicated().sum():
print('Количество явных дубликатов в таблице:', df.duplicated().sum())

Количество явных дубликатов в таблице: 71


In [39]:
#Удаляем явные дубликаты вместе с их индексами (reset_index(drop=True): 
df = df.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов после применения метода drop_duplicates: ', df.duplicated().sum())
print('Количество наблюдений в данных после удаления всех дубликатов: ', df.shape[0])

Количество дубликатов после применения метода drop_duplicates:  0
Количество наблюдений в данных после удаления всех дубликатов:  21453


**Вывод**

<div style="border:solid green 2px; padding: 20px">

В нашем датасете были обнаружены явные и неявные дубликаты. Одна из причин появления: некоторые значения были набраны разными регистрами. Явные дубликаты успешно удалены с помощью цепочки методов drop_duplicates().reset_index(drop=True). Неявные дубликаты - строковые значения в разных регистрах - устранены с помощью метода str.lower(). 

### Лемматизация

In [40]:
#Посмотрим на значения в столбце с целью, чтобы понимать, что будем лемматизировать: 
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Получим леммы в значениях столбца с целями получения кредита

In [41]:
#Создадим функцию, которая лемматизирует (=приводит слова к начальной форме слова) текст, удаляет пробелы и стоп-слова и возращает только полнозначные слова в начальной форме.

"""
Аргумент функции text - тип данных строка.
Функция возращает строку из ключевых лемм - тип данных строка.

"""

#Возьмем список стоп-слов для русского языка из nltk.stopwords:
russian_stopwords = stopwords.words('russian')

def lemmatize_text(text):
    lemmas = mystem.lemmatize(text) #лемматизируем текст
    lemmas_only_key_words = [] #создаем пустой список для лемм полнозначных слов
    for lemma in lemmas:
#если лемма не является пробелом и не принадлежит к числу стоп-слов, то добавляем ее в наш список
        if lemma != ' ' and lemma not in russian_stopwords: 
            lemmas_only_key_words.append(lemma)
            
#склеиваем список в строку, разделяя леммы пробелом
    key_lemmas = ' '.join(lemmas_only_key_words).strip()
    return key_lemmas

In [42]:
#Протестируем функцию на отдельном значении: 
print('Функция работает корректно:\n', lemmatize_text('операции с коммерческой недвижимостью'))

Функция работает корректно:
 операция коммерческий недвижимость


In [43]:
#Применим нашу функцию к столбцу с целями получения кредита (purpose) и запишем значения в новый столбец purpose_words:
df['purpose_words'] = df['purpose'].apply(lemmatize_text)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_words
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,поздний возраст,покупка жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний возраст,приобретение автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,средний возраст,покупка жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,средний возраст,дополнительный образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,поздний возраст,сыграть свадьба


In [44]:
df['purpose_words'].value_counts()

автомобиль                                972
свадьба                                   791
проведение свадьба                        768
сыграть свадьба                           765
операция недвижимость                     675
покупка коммерческий недвижимость         661
операция жилье                            652
покупка жилье сдача                       651
операция коммерческий недвижимость        650
покупка жилье                             646
жилье                                     646
покупка жилье семья                       638
строительство собственный недвижимость    635
недвижимость                              633
операция свой недвижимость                627
строительство жилой недвижимость          624
покупка недвижимость                      620
покупка свой жилье                        620
строительство недвижимость                619
ремонт жилье                              607
покупка жилой недвижимость                606
покупка свой автомобиль           

**Вывод**

<div style="border:solid green 2px; padding: 20px"> 
Мы лемматизировали значения в столбце с целями получения кредита с помощью библиотеку pymystem3 и создали новый столбец purpose_words. На основе этого столбца мы сможем категоризировать данные о цели.  

### Категоризация данных

In [45]:
#Создадим функцию, которая возвращает нам категорию цели получения кредита: 

'''
Функция берет на вход строковое значение, возвращает целевую категорию в зависимости от ключевого существительного.
Там, где возможно классифицировать вид недвижимости (по наличию слов "жилой" и "коммерческий"), мы классифицируем. 
Там где это невозможно - оставляем общий термин "недвижимость".

Аргументы функции: string (тип данных строка)
Функция возвращает строку.

'''

def get_purpose_category (string):
    
    if 'автомобиль' in string:
        return 'автомобиль'
    
    elif 'свадьба' in string:
        return 'свадьба'
    
    elif 'недвижимость' in string and 'коммерческий' in string:
        return 'коммерческая недвижимость'
    
    elif 'жилой' in string and 'недвижимость' in string:
        return 'жилая недвижимость'
    
    elif 'жилье' in string:
        return 'жилая недвижимость'
    
    elif 'образование' in string:
        return 'образование'
    
    else:
        return 'недвижимость'
        

In [46]:
#Протестируем функцию на отдельных значениях:
print(get_purpose_category('покупка свой автомобиль'))
print(get_purpose_category('операция коммерческий недвижимость'))
print(get_purpose_category('покупка жилье сдача'))
print(get_purpose_category('покупка жилой недвижимость'))   
print(get_purpose_category('операция свой недвижимость'))
print(get_purpose_category('сыграть свадьба'))
print(get_purpose_category('получение дополнительный образование'))      

автомобиль
коммерческая недвижимость
жилая недвижимость
жилая недвижимость
недвижимость
свадьба
образование


In [47]:
# Все готово для категоризации данных о целях получения кредита: 
#вычислим целевые категории по ключевым словам из столбца 'purpose' с помощью нашей функции get_purpose_category внутри apply() и запишем новые значения в столбец purpose_category:
df['purpose_category'] = df['purpose_words'].apply(get_purpose_category)

In [48]:
# Проверим, сколько каких значений у нас получилось в новом столбце purpose_category:
df['purpose_category'].value_counts()

жилая недвижимость           5690
автомобиль                   4306
образование                  4013
недвижимость                 3809
свадьба                      2324
коммерческая недвижимость    1311
Name: purpose_category, dtype: int64

In [49]:
#Столбец c ключевыми словами для цели нам более не нужен - удалим его с помощью drop() 
#и сохраним изменения в исходном датафрейме с помощью параметра inplace=True:
df.drop('purpose_words', axis=1, inplace=True)

In [50]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,поздний возраст,жилая недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний возраст,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,средний возраст,жилая недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,средний возраст,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,поздний возраст,свадьба


**Вывод**

<div style="border:solid green 2px; padding: 20px"> 
Данные о целях получения кредита успешно категоризированы. 
    
Мы разделили все цели на 6 категорий: 

* жилая недвижимость
* коммерческая недвижимость
* недвижимость (общая категория для тех случаев, когда не было конкретизации)
* автомобиль
* образование 
* свадьба 

Можно утверждать, что чаще всего люди берут кредиты для вложения в жилую недвижимость, автомобиль и образование.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [51]:
#Для начала посмотрим, какой процент клиентов не выплачивает кредит в срок в наших данных
print("Процент просроченных платежей: {:.1%}".format(df['debt'].mean()))
print('Количество должников:', df['debt'].sum())

Процент просроченных платежей: 8.1%
Количество должников: 1741


In [52]:
# Построим сводную таблицу методом pivot_table, сгруппировав данные по количеству детей и факту задолжности, посчитаем количество клиентов с долгами и без в зависимости от количества детей.
# Среди должников нет клиентов с 5 детьми, поэтому заполним значение NaN с помощью параметра fill_value=0.
# C помощью параметра margins выведем обобщенные значения по каждой группе.
pivot_table_children = df.pivot_table(index=['children'], columns='debt', values='income_type', aggfunc='count',fill_value=0, margins=True)
pivot_table_children 

debt,0,1,All
children,,,
0,13027,1063,14090
1,4410,445,4855
2,1926,202,2128
3,303,27,330
4,37,4,41
5,9,0,9
All,19712,1741,21453


In [53]:
#Для лучшего прочтения таблицы изменим названия столбцов 0, 1 на 'no_debts','with_debts':
pivot_table_children.set_axis(['no_debts','with_debts', 'all'],axis = 'columns',inplace = True)
pivot_table_children 

,no_debts,with_debts,all
children,,,
0,13027,1063,14090
1,4410,445,4855
2,1926,202,2128
3,303,27,330
4,37,4,41
5,9,0,9
All,19712,1741,21453


In [54]:
#Добавим столбец с процентом клиентов, не имеющих задолжность, от общего числа клиентов по каждой группе в зависимости от количества детей.
pivot_table_children['percent_no_debts'] = pivot_table_children['no_debts'] / pivot_table_children['all'] * 100
pivot_table_children['percent_with_debts'] = pivot_table_children['with_debts'] / pivot_table_children['all'] * 100

# Поменяем тип данных на целочисленный (после операции выше он стал дробным)
pivot_table_children['percent_no_debts'] = pivot_table_children['percent_no_debts'].astype('int')
pivot_table_children['percent_with_debts'] = pivot_table_children['percent_with_debts'].astype('int')

In [55]:
pivot_table_children

,no_debts,with_debts,all,percent_no_debts,percent_with_debts
children,,,,,
0,13027,1063,14090,92,7
1,4410,445,4855,90,9
2,1926,202,2128,90,9
3,303,27,330,91,8
4,37,4,41,90,9
5,9,0,9,100,0
All,19712,1741,21453,91,8


In [56]:
#Столбец c обобщенными данными нам больше не нужен - удалим его с помощью drop() и отсортируем значения столбца percent_no_debts по убыванию:
pivot_table_children = pivot_table_children.drop('all', axis=1).sort_values(by='percent_no_debts', ascending=False)

In [57]:
#Посмотрим на результаты:
pivot_table_children

,no_debts,with_debts,percent_no_debts,percent_with_debts
children,,,,
5,9,0,100,0
0,13027,1063,92,7
3,303,27,91,8
All,19712,1741,91,8
1,4410,445,90,9
2,1926,202,90,9
4,37,4,90,9


**Вывод**

<div style="border:solid green 2px; padding: 20px"> 

Наличие ребенка влияет негативным образом на факт погашения кредита в срок. Чаще всего кредит возращают в срок люди, у которых нет детей. И среди должников наблюдаем наименьший процент тех, у кого детей нет. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [58]:
#Построим сводную таблицу, сгруппировав клиентов в зависимости от семейного положения и факта задолжности
pivot_table_family = df.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count', margins=True)

#Изменим названия столбцов 0, 1:
pivot_table_family.set_axis(['no_debts','with_debts', 'all'],axis = 'columns',inplace = True)

In [59]:
pivot_table_family

,no_debts,with_debts,all
family_status,,,
в разводе,1110,85,1195
вдовец / вдова,896,63,959
гражданский брак,3762,388,4150
женат / замужем,11408,931,12339
не женат / не замужем,2536,274,2810
All,19712,1741,21453


In [60]:
#Добавим столбцы с процентом клиентов, не имеющих задолжность и имеющих, от общего числа клиентов по каждой группе в зависимости от семейного положения.
pivot_table_family['percent_no_debts'] = pivot_table_family['no_debts'] / pivot_table_family['all'] * 100
pivot_table_family['percent_with_debts'] = pivot_table_family['with_debts'] / pivot_table_family['all'] * 100

#Поменяем тип данных в новых столбцах
pivot_table_family['percent_no_debts'] = pivot_table_family['percent_no_debts'].astype('int')
pivot_table_family['percent_with_debts'] = pivot_table_family['percent_with_debts'].astype('int')

In [61]:
#Столбец c обобщенными данными нам больше не нужен - удалим его с помощью drop() и отсортируем значения столбца percent_no_debts по убыванию:
pivot_table_family = pivot_table_family.drop('all', axis=1).sort_values(by='percent_no_debts', ascending=False)

In [62]:
#Смотрим результат
pivot_table_family

,no_debts,with_debts,percent_no_debts,percent_with_debts
family_status,,,,
вдовец / вдова,896,63,93,6
в разводе,1110,85,92,7
женат / замужем,11408,931,92,7
All,19712,1741,91,8
гражданский брак,3762,388,90,9
не женат / не замужем,2536,274,90,9


**Вывод**

<div style="border:solid green 2px; padding: 20px"> 
Чаще всего кредит возвращают в срок вдовцы, разведенные и женатые и замужние, а не замужние и не женатые чаще всего имеют задолжность. Любопытно, что ответственность за принятие решение - узаконить отношения как будто бы влияет на платежеспособность


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Прежде нам нужно разбить клиентов на группы в зависимости от уровня ежемесячного дохода, тогда мы сможем сравнить каждую группу и сделать выводы. 
Размах значений большой: "разрежем" данные ежемесячного дохода на шесть интервалов: группа с максимально низким и группа с максимально высоким доходом (по 10% данных каждая), остальные группы с одинаковым количеством элементов. Границы относительно всех доходов: [10, 30, 50, 70, 90, 100].

In [63]:
#Разделим данные столбца total_income на 6 интервалов (параметр q) с помощью метода qcut:
pd.qcut(df['total_income'], q=[0,.1,.3,.5,.7,.9,1]).value_counts()

(116007.4, 142594.0]     4407
(78719.4, 116007.4]      4290
(179797.2, 269829.0]     4290
(142594.0, 179797.2]     4174
(20666.999, 78719.4]     2146
(269829.0, 2265604.0]    2146
Name: total_income, dtype: int64

In [64]:
#Создадим столбец income_group с интервалами ежемесячного дохода
df['income_group'] = pd.qcut(df['total_income'], q=[0,.1,.3,.5,.7,.9,1])
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_category,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,поздний возраст,жилая недвижимость,"(179797.2, 269829.0]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний возраст,автомобиль,"(78719.4, 116007.4]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,средний возраст,жилая недвижимость,"(142594.0, 179797.2]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,средний возраст,образование,"(179797.2, 269829.0]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,поздний возраст,свадьба,"(142594.0, 179797.2]"


In [65]:
#С помощью метода pivot_table сделаем сводную таблицу:
pivot_table_income = df.pivot_table(index='income_group', columns='debt', values='family_status_id', aggfunc='count', margins=True)

#Изменим названия столбцов 0, 1:
pivot_table_income.set_axis(['no_debts','with_debts', 'all'],axis = 'columns',inplace = True)


In [66]:
pivot_table_income

,no_debts,with_debts,all
income_group,,,
"(20666.999, 78719.4]",1989,157,2146
"(78719.4, 116007.4]",3924,366,4290
"(116007.4, 142594.0]",4020,387,4407
"(142594.0, 179797.2]",3824,350,4174
"(179797.2, 269829.0]",3960,330,4290
"(269829.0, 2265604.0]",1995,151,2146
All,19712,1741,21453


In [67]:
#Добавим столбцы с процентом клиентов, не имеющих задолжность и имеющих, от общего числа клиентов по каждой группе.
pivot_table_income['percent_no_debts'] = pivot_table_income['no_debts'] / pivot_table_income['all'] * 100
pivot_table_income['percent_with_debts'] = pivot_table_income['with_debts'] / pivot_table_income['all'] * 100

#Поменяем тип данных в новых столбцах
pivot_table_income['percent_no_debts'] = pivot_table_income['percent_no_debts'].astype('int')
pivot_table_income['percent_with_debts'] = pivot_table_income['percent_with_debts'].astype('int')

In [68]:
#Столбец c обобщенными данными нам больше не нужен - удалим его с помощью drop() и отсортируем значения столбца percent_no_debts по убыванию:
pivot_table_income = pivot_table_income.drop('all', axis=1).sort_values(by='percent_no_debts', ascending=False)

In [69]:
pivot_table_income 

,no_debts,with_debts,percent_no_debts,percent_with_debts
income_group,,,,
"(20666.999, 78719.4]",1989,157,92,7
"(179797.2, 269829.0]",3960,330,92,7
"(269829.0, 2265604.0]",1995,151,92,7
"(78719.4, 116007.4]",3924,366,91,8
"(116007.4, 142594.0]",4020,387,91,8
"(142594.0, 179797.2]",3824,350,91,8
All,19712,1741,91,8


**Вывод**
<div style="border:solid green 2px; padding: 20px"> 
Анализ данных в сводной таблице показал неожиданный результат: люди с наименьшим доходом возвращают кредит в срок так же, как и люди с самым высоким доходом. 

- Как разные цели кредита влияют на его возврат в срок?

In [70]:
#Создадим сводную таблицу, в которой напротив каждой целевой категории будет указано, сколько клиентов с долгами и без
pivot_table_purpose = df.pivot_table(index='purpose_category', columns='debt', values='family_status_id', aggfunc='count', margins=True)

#Изменим названия столбцов 0, 1:
pivot_table_purpose.set_axis(['no_debts','with_debts', 'all'],axis = 'columns',inplace = True)

In [71]:
pivot_table_purpose

,no_debts,with_debts,all
purpose_category,,,
автомобиль,3903,403,4306
жилая недвижимость,5293,397,5690
коммерческая недвижимость,1212,99,1311
недвижимость,3523,286,3809
образование,3643,370,4013
свадьба,2138,186,2324
All,19712,1741,21453


In [72]:
#Добавим столбцы с процентом клиентов, не имеющих задолжность и имеющих, от общего числа клиентов по каждой группе 
pivot_table_purpose['percent_no_debts'] = pivot_table_purpose['no_debts'] / pivot_table_purpose['all'] * 100
pivot_table_purpose['percent_with_debts'] = pivot_table_purpose['with_debts'] / pivot_table_purpose['all'] * 100

#Поменяем тип данных в новых столбцах
pivot_table_purpose['percent_no_debts'] = pivot_table_purpose['percent_no_debts'].astype('int')
pivot_table_purpose['percent_with_debts'] = pivot_table_purpose['percent_with_debts'].astype('int')

In [73]:
#Столбец c обобщенными данными нам больше не нужен - удалим его с помощью drop() и отсортируем значения столбца percent_no_debts по убыванию:
pivot_table_purpose = pivot_table_purpose.drop('all', axis=1).sort_values(by='percent_no_debts', ascending=False)

In [74]:
pivot_table_purpose

,no_debts,with_debts,percent_no_debts,percent_with_debts
purpose_category,,,,
жилая недвижимость,5293,397,93,6
коммерческая недвижимость,1212,99,92,7
недвижимость,3523,286,92,7
свадьба,2138,186,91,8
All,19712,1741,91,8
автомобиль,3903,403,90,9
образование,3643,370,90,9


**Вывод**

<div style="border:solid green 2px; padding: 20px"> 
Чаще всего клиенты возвращают кредит в срок, если берут его с целью вложиться в недвижимость: те, кто вкладывает в жилую недвижимость, - самые надежные плательщики. А самыми ненадежными являются те, кто берет кредит на образование. 

In [75]:
#В конце исследования еще раз посмотрим на наш датасет, чтобы убедиться, что он в нужном (очищенном и обогащенном) состоянии. 
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_category,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,поздний возраст,жилая недвижимость,"(179797.2, 269829.0]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний возраст,автомобиль,"(78719.4, 116007.4]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,средний возраст,жилая недвижимость,"(142594.0, 179797.2]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,средний возраст,образование,"(179797.2, 269829.0]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,поздний возраст,свадьба,"(142594.0, 179797.2]"


In [76]:
# И сохраним его для возможных дальнейших исследований:
df.to_csv('bank_clients_mortgage.csv')

Для операций со сводными таблицами можно создать отдельную функцию, чтобы оптимизировать код в данном разделе. 

## Шаг 4. Общий вывод

В данном исследовании нам нужно было ответить на 4 вопроса: 


1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

4. Как разные цели кредита влияют на его возврат в срок?


Мы предобработали данные, построили сводные таблицы по этим очищенным данным, провели анализ полученных данных и установили:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок? 

Есть. И зависимость такова: факт наличия детей уменьшает вероятность возвращения кредита в срок. 


2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Есть. Чаще всего кредит возвращают в срок вдовцы, разведенные и женатые и замужние, а не замужние и не женатые чаще всего имеют задолжность.


3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

По имеющимся данным зависимости нет. Плательщики с наименьшим ежемесячным доходом так же часто возвращают кредит в срок, как и люди с самым большим доходом. В данных о ежемесячном доходе были пропуски и, возможно, если бы не пропуски, результат был бы иной.  


4. Как разные цели кредита влияют на его возврат в срок?

Чаще всего кредит выплачивают, если он взят с целью вложения в: 1) жилую недвижимость, 2) коммерческую недвижимость, 3) свадьбу. И чаще всего не выплачиваются в срок кредиты на образование.